Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [2]:
NAME = "SIDDHARTH YASHASWEE"
COLLABORATORS = ""

---

# Shape Descriptors
In this notebook we will implement the Laplace Beltrami operator and use it to compute the mean curvature and normals as well as the heat kernel signature for an example mesh.

In [3]:
import numpy as np
import openmesh as om
from scipy.sparse import csr_matrix, diags
from scipy.sparse.linalg import eigsh
import k3d

In [4]:
mesh = om.read_trimesh("spot_triangulated.obj")

## Laplace-Beltrami
In this task you are going to implement the Laplace-Beltrami operator.
In the lecture we have described this operator as a sum, however in practice it usually makes more sense to write it as a matrix. This matrix can then e.g. be multiplied with the vertex positions to obtain the Discrete Mean Curvature Normal operator.
This matrix can be split into two parts: 
- The mass matrix contains the vertex area
- The (weak) Laplace matrix contains the cotangent weights

In the following tasks you will build these two matrices.
We have implemented two helper functions for you:
- `triangle_area` computes the area of a triangle, given its three vertex positions
- `cotangent_weight` computes the cotangent weight for a given edge. (In the lecture slides you see a factor of 1/2 before the sum, this factor is part of the cotangent weights)

In previous exercises you have seen how to use the functions `diags` and `csr_matrix` to construct sparse matrices.\
In the introduction we introduced you to openmesh and have shown you how to obtain vertex coordinates, face-vertex indices and iterate over vertices or a vertex neigbourhood. Similarly you can iterate over faces, with `mesh.faces()` or obtain the faces, neighbouring a vertex with `mesh.vf(v)`
The id of a face is obtained from a facee handle with `idx()` (similarly as for vertices).\
Furthermore, for this exercise you will need to get the two vertices of an edge. 
These can be obtained with `mesh.to_vertex_handle(mesh.halfedge_handle(eh,0))`. Replace 0 with 1 to obtain the other vertex.

In [5]:
# helper function: given triangle vertices, compute area
def triangle_area(a,b,c):
    return np.linalg.norm(np.cross(b-a,c-a))/2
    
def mass_matrix(mesh):
    ### YOUR CODE HERE
    
    v_area_matrix = []
    pts = mesh.points()
    faces = mesh.fv_indices()

    for vh in mesh.vertices():
        v_area = 0
        
        #iterate over all outgoing halfedges
        for oh in mesh.voh(vh):
            #face handle for halfedge
            fh = mesh.face_handle(oh)
            face_indices = faces[fh.idx()]
            face_pts = pts[face_indices]
            #calc area again and again is inefficient, store all areas for faces and 
            #access as required
            area = triangle_area(face_pts[0], face_pts[1], face_pts[2])
            v_area += area
        
        v_area = v_area/3
        v_area_matrix.append(v_area)
    #print(diags(v_area_matrix))
    return diags(v_area_matrix)

# helper function: given edge, compute cotangent weight
def cotangent_weight(mesh, eh):
    v0 = mesh.point(mesh.to_vertex_handle(mesh.halfedge_handle(eh,0)))
    v1 = mesh.point(mesh.to_vertex_handle(mesh.halfedge_handle(eh,1)))
    weight = 0
    for i in range(2):
        heh = mesh.halfedge_handle(eh,i)
        opp_v = mesh.point(mesh.to_vertex_handle(mesh.next_halfedge_handle(heh)))
        dir1 = opp_v - v0
        dir2 = opp_v - v1
        weight += np.dot(dir1,dir2) / np.linalg.norm(np.cross(dir1,dir2))
    return weight / 2

def laplace_matrix(mesh):
    ### YOUR CODE HERE - solved from Discrete laplace Wikipedia
    #see official soln for an faster computation
    n_v = len(mesh.points())
    l_mat = csr_matrix((n_v, n_v))
    
    for vh in mesh.vertices():       
        #iterate over all outgoing halfedges
        for oh in mesh.voh(vh): 
            eh = mesh.edge_handle(oh)
            weight = cotangent_weight(mesh, eh)
            
            vi_idx = mesh.from_vertex_handle(oh).idx()
            vj_idx = mesh.to_vertex_handle(oh).idx()
            
            l_mat[vi_idx, vj_idx] = weight
            
    #now need to do the diag elements
    for vh in mesh.vertices():        
        d_entry = 0
        #iterate over all outgoing halfedges
        for oh in mesh.voh(vh): 
            vi_idx = mesh.from_vertex_handle(oh).idx()
            vj_idx = mesh.to_vertex_handle(oh).idx()
            
            l_mat[vi_idx, vi_idx] -= l_mat[vi_idx, vj_idx]

    return l_mat

def invert_diagonal_matrix(D):
    d = D.diagonal()
    zero_mask = d == 0
    inv_m = diags(np.where(zero_mask, 1, 1/d))
    #print(inv_m.shape) 2930x2930
    #print(type(inv_m)) - dia_matrix
    return inv_m



In [6]:
M = mass_matrix(mesh)
M_inv = invert_diagonal_matrix(M)
L = laplace_matrix(mesh)
hn = M_inv.dot(L.dot(mesh.points()))
h = np.linalg.norm(hn, axis=1)
normals = hn / h[:,None]

C:\Users\lenovo1\.conda\envs\shapeanalysis\lib\site-packages\scipy\sparse\_index.py:103: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


In the following plot you should see curved areas colored in yellow and flat areas colored in purple.
Furthermore the normals should be orthogonal to the surface and point outwards in convex areas.

In [7]:
plot = k3d.plot()
plot += k3d.mesh(mesh.points(), mesh.fv_indices(), attribute=h/2, 
                 color_map=k3d.colormaps.matplotlib_color_maps.viridis,
                color_range=(0,10))
plot += k3d.vectors(mesh.points(), normals/10, line_width=0.0001, use_head=False)
plot

C:\Users\lenovo1\.conda\envs\shapeanalysis\lib\site-packages\traittypes\traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(
C:\Users\lenovo1\.conda\envs\shapeanalysis\lib\site-packages\traittypes\traittypes.py:97: UserWarning: Given trait value dtype "int32" does not match required type "uint32". A coerced copy has been created.
  warnings.warn(


Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

In [8]:
np.testing.assert_array_almost_equal(M.todense()[range(5),range(5)], 
                                     [[0.00478942, 0.00284357, 0.00094711, 0.0006429, 0.00267439]], decimal=8)
### BEGIN HIDDEN TESTS
np.testing.assert_array_almost_equal(M.todense()[range(100,105),range(100,105)], 
           [[0.00019515, 0.00081228, 0.0005961, 0.0008419, 0.00028521]], decimal=8)
### END HIDDEN TESTS

In [9]:
np.testing.assert_approx_equal(L.todense()[0,0], -4.230992097869423, significant=8)
np.testing.assert_approx_equal(L.todense()[0,812], 1.4068019986623206, significant=8)
np.testing.assert_approx_equal(L.todense()[729,2923], 1.0595635160548063, significant=8)
### BEGIN HIDDEN TESTS
np.testing.assert_approx_equal(L.todense()[1586,1586], -5.541093138478171, significant=8)
np.testing.assert_approx_equal(L.todense()[1586,412], 0.08702211729571252, significant=8)
np.testing.assert_approx_equal(L.todense()[2923,729], 1.0595635160548063, significant=8)
### END HIDDEN TESTS

## Heat Kernel Signature
In this task you will use the previously defined Laplace-Beltrami operator to compute the Heat Kernel Signature.
You can compute the eigenvalues and eigenvectors using scipy with the function  call `eigsh(L, n_eig, M, sigma=0)`.
Besides the previously computed (weak) Laplacian and mass matrix, the function `heat_kernel_signature(L, M, n_eig, t)` gets as input the number of eigenvalues we will use, and the time step.
You can check your result by trying out several values for the time step

In [96]:
def heat_kernel_signature(L, M, n_eig, t):
    ### YOUR CODE HERE
    #M is mass matrix
    #n_eig The number of eigenvalues and eigenvectors desired.
    #If M is specified, solves A @ x[i] = w[i] * M @ x[i], 
    #the generalized eigenvalue problem for w[i] eigenvalues with corresponding eigenvectors x[i].
    w,v = eigsh(L, n_eig, M, sigma=0)
#     print(w.shape)
#     print(v.shape)
    
    #not sure abt this, verify if used
    w_vals = np.exp(-1*abs(w)*t)
    #print(w_vals.shape)
    v_vals = np.multiply(v,v)
    #print(v_vals.shape[0])
    mat = np.zeros(v_vals.shape[0])
                       
    for i in range(len(w_vals)) :
        mat += w_vals[i]*v_vals[:, i]
        
#     print(w[None, :].shape)
    
    return mat

In [93]:
vals = heat_kernel_signature(L,M,100,1e-3)

In [94]:
plot = k3d.plot()
plot += k3d.mesh(mesh.points(), mesh.fv_indices(), attribute=vals, 
                 color_map=k3d.colormaps.matplotlib_color_maps.viridis)
plot

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

In [95]:
vals = heat_kernel_signature(L,M,100,1e-3)
np.testing.assert_approx_equal(vals.min(), 9.59263418573553, significant=8)
np.testing.assert_approx_equal(vals.max(), 30.681926901103605, significant=8)